In [1]:
##Import packages needed (and some not needed....)

import Bio
import openpyxl
from Bio import SeqIO
import os,sys,re
from Bio.Seq import Seq
import pandas as pd
print(pd.__name__, pd.__version__)

import numpy as np
print(np.__name__, np.__version__)

pandas 1.3.4
numpy 1.20.3


In [2]:
#set-up DNA sequence and codons

#wt sequence
CaERG11 = "ATGGCCATTGTTGAAACGGTAATAGATGGAATCAACTATTTTTTATCACTGTCTGTAACTCAACAAATTAGTATATTACTAGGGGTCCCGTTTGTGTACAATCTGGTTTGGCAGTATTTATATAGTCTAAGAAAAGATAGAGCACCCCTGGTATTCTATTGGATACCCTGGTTCGGCTCAGCAGCGTCTTATGGTCAACAACCGTATGAATTTTTTGAGTCATGCAGGCAGAAATATGGCGATGTTTTTTCCTTTATGCTTCTTGGGAAAATAATGACTGTGTACCTTGGCCCAAAGGGTCACGAATTTGTCTTCAATGCGAAGTTAAGCGATGTCTCTGCTGAAGATGCATACAAACACTTAACAACTCCGGTATTCGGAAAGGGAGTTATCTACGATTGTCCCAACAGTCGTCTGATGGAGCAGAAAAAGTTTGCTAAATTCGCGCTTACTACGGACTCATTCAAAAGGTATGTACCCAAAATTCGTGAAGAAATATTAAACTATTTCGTAACAGACGAAAGTTTCAAATTAAAGGAAAAGACACACGGCGTGGCTAACGTGATGAAAACACAACCTGAAATCACCATTTTTACCGCCAGCAGGTCCTTATTTGGCGACGAGATGAGAAGAATTTTCGATAGAAGTTTTGCTCAACTGTACTCAGATTTGGATAAAGGTTTTACTCCGATAAATTTCGTGTTCCCAAACCTTCCTTTGCCACACTATTGGAGAAGAGATGCTGCACAAAAGAAGATAAGTGCGACCTATATGAAAGAGATAAAATCCCGTAGGGAAAGGGGTGACATTGACCCTAACAGAGACTTGATTGATTCTCTGCTGATTCATAGCACATACAAAGACGGTGTGAAGATGACCGACCAGGAGATTGCAAACCTGCTTATAGGTATTTTAATGGGTGGACAGCACACATCAGCTTCTACGTCAGCATGGTTTCTGCTACACCTGGGGGAAAAACCGCACCTACAAGACGTAATATATCAAGAGGTGGTGGAGTTACTAAAAGAGAAAGGAGGCGATCTAAACGATTTAACCTATGAGGATCTGCAAAAATTGCCGTCAGTCAACAACACGATCAAGGAAACGCTAAGGATGCACATGCCTCTACACAGTATATTCAGAAAAGTTACTAACCCACTTAGGATCCCTGAAACCAACTACATCGTCCCAAAAGGACACTACGTTCTTGTCAGCCCAGGCTACGCACACACGAGTGAGAGATACTTTGATAACCCGGAGGATTTTGATCCTACACGTTGGGATACTGCTGCAGCCAAAGCCAATTCTGTAAGCTTTAACTCCAGTGATGAGGTAGATTACGGCTTTGGGAAAGTATCAAAAGGCGTCAGCTCACCATATCTTCCCTTCGGTGGCGGTAGACATAGATGTATAGGTGAACAATTTGCATACGTTCAGCTGGGAACCATATTAACGACGTTTGTTTATAACTTGAGATGGACTATCGACGGGTATAAAGTCCCTGATCCTGACTATAGCTCTATGGTTGTTCTACCCACCGAACCGGCTGAAATAATTTGGGAGAAAAGAGAGACATGTATGTTTTGA"

DNA_Codons = {"GCT": "A", "GCC": "A", "GCA": "A", "GCG": "A","TGT": "C", "TGC": "C",
    "GAT": "D", "GAC": "D","GAA": "E", "GAG": "E","TTT": "F", "TTC": "F",
    "GGT": "G", "GGC": "G", "GGA": "G", "GGG": "G","CAT": "H", "CAC": "H","ATA": "I", "ATT": "I", "ATC": "I",
    "AAA": "K", "AAG": "K", "TTA": "L", "TTG": "L", "CTT": "L", "CTC": "L", "CTA": "L", "CTG": "L",
    "ATG": "M","AAT": "N", "AAC": "N","CCT": "P", "CCC": "P", "CCA": "P", "CCG": "P","CAA": "Q", "CAG": "Q",
    "CGT": "R", "CGC": "R", "CGA": "R", "CGG": "R", "AGA": "R", "AGG": "R","TCT": "S", "TCC": "S", "TCA": "S", 
    "TCG": "S", "AGT": "S", "AGC": "S","ACT": "T", "ACC": "T", "ACA": "T", "ACG": "T",
    "GTT": "V", "GTC": "V", "GTA": "V", "GTG": "V","TGG": "W","TAT": "Y", "TAC": "Y",
    "TAA": "*", "TAG": "*", "TGA": "*"} #all codons + corresponding aa

codons_CaERG11 = {"GCT": "A", "TGT": "C","GAT": "D","GAA": "E", "TTC": "F","GGT": "G","CAC": "H",
    "ATT": "I", "AAG": "K", "TTG": "L","ATG": "M","AAC": "N","CCA": "P","CAA": "Q",
    "AGA": "R", "TCT": "S", "ACT": "T","GTT": "V", "TGG": "W", "TAC": "Y"} #codons + corresponding aa I use for the mutagenesis

variants = ["57F", "112K", "113I", "131G", "132N", "132M", "143L", "230L", "292I", "292R", "303P", "311Q", "363M",
            "375C", "379K", "464V", "476N", "506T", "60Y", "64W", "88F", "128H", "130L", "146H", "280R", "302A",
            "464W", "468M", "506V", "105A", "154W", "277L", "314I", "465Q", "467S", "509M", "59K", "147C", "203V",
            "312R", "480K", "509Y", "67F", "103L", "151R", "362C", "458R", "477V"] #list of the variants I want to get

pos_list = []
aa_list = []

#list of the positions and the amino acids individually using the variants list
for variant in variants:
    number, letter = variant[:-1], variant[-1]
    pos_list.append(number)
    aa_list.append(letter)

print(aa_list)
print(pos_list)

['F', 'K', 'I', 'G', 'N', 'M', 'L', 'L', 'I', 'R', 'P', 'Q', 'M', 'C', 'K', 'V', 'N', 'T', 'Y', 'W', 'F', 'H', 'L', 'H', 'R', 'A', 'W', 'M', 'V', 'A', 'W', 'L', 'I', 'Q', 'S', 'M', 'K', 'C', 'V', 'R', 'K', 'Y', 'F', 'L', 'R', 'C', 'R', 'V']
['57', '112', '113', '131', '132', '132', '143', '230', '292', '292', '303', '311', '363', '375', '379', '464', '476', '506', '60', '64', '88', '128', '130', '146', '280', '302', '464', '468', '506', '105', '154', '277', '314', '465', '467', '509', '59', '147', '203', '312', '480', '509', '67', '103', '151', '362', '458', '477']


In [3]:
#fonction to get a key from a dictionnary when you have the value

def get_key(val,my_dict):
   
    for key, value in my_dict.items():
        if val == value:
            return key
 
    return "key doesn't exist"

In [21]:
#function to get 2 df, one with the forward primers and one with the reverse primers

def get_mut_oligo(gene, sequence, variants, DNA_Codons, codons_mut):
    
    list_forward = []
    list_reverse = []
    list_comment = []
    df_forward = []
    df_reverse = []
    list_variants = []
    DNA_variants = []
    
    #Get the codons in the wt seq and make a dictionnary out of it

    wt_dict = {}
    pos = 1

    for i in range(0, len(sequence), 3):
        codon = sequence[i:i + 3]
        wt_dict.update({pos : codon})
        pos = pos+1
        
    #Get the codon you want to mutate and the corresponding primer (40 nt long, 19 + 3 + 18)
    
    for i in range(len(variants)):
        
        position = pos_list[i]
        print("Variant pos = " + str(pos_list[i]))
        amino_acid = aa_list[i]
        
        codon = wt_dict[int(position)] #get the wt codon
        print("WT codon = " + codon)
        
        aa = DNA_Codons[codon] #get the wt aa
        print("WT aa = " + aa)
        
        new_codon = get_key(amino_acid,codons_mut) #get the new codon
        print("New codon = " + new_codon)
        
        mutated_amino_acid = codons_mut[new_codon] #get the mutated aa
        print("New aa = " + amino_acid)
        
        start = (int(position)*3)-3 #get the number of nt before and after the codon
        end = start+3
        
        mutated_sequence = sequence[:start] + new_codon.lower() + sequence[end:] #get the sequence with the mutated codon
        
        mutated_codon_start = start - 19
        mutated_codon_end = end + 18
        
        forward = mutated_sequence[mutated_codon_start:mutated_codon_end] #get the forward oligo
        
        seq = Seq(forward)
    
        reverse = seq.reverse_complement() #get the reverse oligo
        
        #add everything to the lists
        list_forward.append(forward)
        list_reverse.append(str(reverse))
        
       
        list_variants.append(aa + position +  mutated_amino_acid)
        DNA_variants.append(codon + position + new_codon)
        print(DNA_variants)
        print("-------------------------------")
        
    list_commentF = [com + ", F" for com in list_comment]
    list_commentR = [com + ", R" for com in list_comment]
    
    #lists to df
    df_forward = pd.DataFrame(list_forward, columns=['forward'])
    df_forward["DNA variants"] = DNA_variants
    df_forward["amino acid variants"] = list_variants
    
    df_reverse = pd.DataFrame(list_reverse, columns=['reverse'])
    df_reverse["DNA variants"] = DNA_variants
    df_reverse["amino acid variants"] = list_variants
    
    return df_forward, df_reverse

In [22]:
df_for, df_rev = get_mut_oligo("CaERG11",CaERG11, variants, DNA_Codons, codons_CaERG11)

Variant pos = 57
WT codon = TGG
WT aa = W
New codon = TTC
New aa = F
['TGG57TTC']
-------------------------------
Variant pos = 112
WT codon = GTC
WT aa = V
New codon = AAG
New aa = K
['TGG57TTC', 'GTC112AAG']
-------------------------------
Variant pos = 113
WT codon = TCT
WT aa = S
New codon = ATT
New aa = I
['TGG57TTC', 'GTC112AAG', 'TCT113ATT']
-------------------------------
Variant pos = 131
WT codon = ATC
WT aa = I
New codon = GGT
New aa = G
['TGG57TTC', 'GTC112AAG', 'TCT113ATT', 'ATC131GGT']
-------------------------------
Variant pos = 132
WT codon = TAC
WT aa = Y
New codon = AAC
New aa = N
['TGG57TTC', 'GTC112AAG', 'TCT113ATT', 'ATC131GGT', 'TAC132AAC']
-------------------------------
Variant pos = 132
WT codon = TAC
WT aa = Y
New codon = ATG
New aa = M
['TGG57TTC', 'GTC112AAG', 'TCT113ATT', 'ATC131GGT', 'TAC132AAC', 'TAC132ATG']
-------------------------------
Variant pos = 143
WT codon = AAA
WT aa = K
New codon = TTG
New aa = L
['TGG57TTC', 'GTC112AAG', 'TCT113ATT', 'ATC131

In [24]:
df_for

,forward,DNA variants,amino acid variants
0,GGTATTCTATTGGATACCCttcTTCGGCTCAGCAGCGTCT,TGG57TTC,W57F
1,CAATGCGAAGTTAAGCGATaagTCTGCTGAAGATGCATAC,GTC112AAG,V112K
2,TGCGAAGTTAAGCGATGTCattGCTGAAGATGCATACAAA,TCT113ATT,S113I
3,GGTATTCGGAAAGGGAGTTggtTACGATTGTCCCAACAGT,ATC131GGT,I131G
4,ATTCGGAAAGGGAGTTATCaacGATTGTCCCAACAGTCGT,TAC132AAC,Y132N
5,ATTCGGAAAGGGAGTTATCatgGATTGTCCCAACAGTCGT,TAC132ATG,Y132M
6,CAGTCGTCTGATGGAGCAGttgAAGTTTGCTAAATTCGCG,AAA143TTG,K143L
7,TTTGGATAAAGGTTTTACTttgATAAATTTCGTGTTCCCA,CCG230TTG,P230L
8,ATACAAAGACGGTGTGAAGattACCGACCAGGAGATTGCA,ATG292ATT,M292I
9,ATACAAAGACGGTGTGAAGagaACCGACCAGGAGATTGCA,ATG292AGA,M292R


In [25]:
df_rev

,reverse,DNA variants,amino acid variants
0,AGACGCTGCTGAGCCGAAgaaGGGTATCCAATAGAATACC,TGG57TTC,W57F
1,GTATGCATCTTCAGCAGActtATCGCTTAACTTCGCATTG,GTC112AAG,V112K
2,TTTGTATGCATCTTCAGCaatGACATCGCTTAACTTCGCA,TCT113ATT,S113I
3,ACTGTTGGGACAATCGTAaccAACTCCCTTTCCGAATACC,ATC131GGT,I131G
4,ACGACTGTTGGGACAATCgttGATAACTCCCTTTCCGAAT,TAC132AAC,Y132N
5,ACGACTGTTGGGACAATCcatGATAACTCCCTTTCCGAAT,TAC132ATG,Y132M
6,CGCGAATTTAGCAAACTTcaaCTGCTCCATCAGACGACTG,AAA143TTG,K143L
7,TGGGAACACGAAATTTATcaaAGTAAAACCTTTATCCAAA,CCG230TTG,P230L
8,TGCAATCTCCTGGTCGGTaatCTTCACACCGTCTTTGTAT,ATG292ATT,M292I
9,TGCAATCTCCTGGTCGGTtctCTTCACACCGTCTTTGTAT,ATG292AGA,M292R


In [29]:
# Standardize column names to lowercase
df_for.columns = df_for.columns.str.lower()
df_rev.columns = df_rev.columns.str.lower()

# Rename the column in the second DataFrame to match the first one
df_rev = df_rev.rename(columns={"amino acids variants": "amino acid variants"})

# Merge on both "amino acid variants" and "dna variants" columns
merged_df = pd.merge(df_for, df_rev, on=["amino acid variants", "dna variants"], how="left")


In [30]:
merged_df

,forward,dna variants,amino acid variants,reverse
0,GGTATTCTATTGGATACCCttcTTCGGCTCAGCAGCGTCT,TGG57TTC,W57F,AGACGCTGCTGAGCCGAAgaaGGGTATCCAATAGAATACC
1,CAATGCGAAGTTAAGCGATaagTCTGCTGAAGATGCATAC,GTC112AAG,V112K,GTATGCATCTTCAGCAGActtATCGCTTAACTTCGCATTG
2,TGCGAAGTTAAGCGATGTCattGCTGAAGATGCATACAAA,TCT113ATT,S113I,TTTGTATGCATCTTCAGCaatGACATCGCTTAACTTCGCA
3,GGTATTCGGAAAGGGAGTTggtTACGATTGTCCCAACAGT,ATC131GGT,I131G,ACTGTTGGGACAATCGTAaccAACTCCCTTTCCGAATACC
4,ATTCGGAAAGGGAGTTATCaacGATTGTCCCAACAGTCGT,TAC132AAC,Y132N,ACGACTGTTGGGACAATCgttGATAACTCCCTTTCCGAAT
5,ATTCGGAAAGGGAGTTATCatgGATTGTCCCAACAGTCGT,TAC132ATG,Y132M,ACGACTGTTGGGACAATCcatGATAACTCCCTTTCCGAAT
6,CAGTCGTCTGATGGAGCAGttgAAGTTTGCTAAATTCGCG,AAA143TTG,K143L,CGCGAATTTAGCAAACTTcaaCTGCTCCATCAGACGACTG
7,TTTGGATAAAGGTTTTACTttgATAAATTTCGTGTTCCCA,CCG230TTG,P230L,TGGGAACACGAAATTTATcaaAGTAAAACCTTTATCCAAA
8,ATACAAAGACGGTGTGAAGattACCGACCAGGAGATTGCA,ATG292ATT,M292I,TGCAATCTCCTGGTCGGTaatCTTCACACCGTCTTTGTAT
9,ATACAAAGACGGTGTGAAGagaACCGACCAGGAGATTGCA,ATG292AGA,M292R,TGCAATCTCCTGGTCGGTtctCTTCACACCGTCTTTGTAT


In [31]:
merged_df.to_csv(f'Oligos_validations_2023-07-13.csv')  